In [3]:
import pandas as pd, numpy as np

df = pd.read_csv("1st-dataset.csv")     # 1차 데이터셋
pd.set_option("display.max_columns", None)

# 분석용 편의 함수
def pct(n): return f"{n:.1%}"

In [4]:
# 결측치 체크

na_cnt   = df.isna().sum()
na_rate  = (na_cnt/len(df)).map(pct)
na_report = pd.concat([na_cnt, na_rate], axis=1, keys=["건수", "비율"])
print(na_report.sort_values("비율", ascending=False))

                건수    비율
game_id          0  0.0%
audience         0  0.0%
precipitation    0  0.0%
temperature      0  0.0%
death_count      0  0.0%
injury_count     0  0.0%
accident_count   0  0.0%
region           0  0.0%
match_type       0  0.0%
stadium_code     0  0.0%
home_team_win    0  0.0%
start_time       0  0.0%
is_holiday       0  0.0%
day_of_week      0  0.0%
game_date        0  0.0%
sports_type      0  0.0%
snow_depth       0  0.0%


In [26]:

# 기본 치환: '페넌트레이스'·'순위결정전'(오탈자 '순위결정정' 포함)·'0' --> '정규시즌'
df["match_type"] = (
    df["match_type"]
        .replace({"페넌트레이스": "정규시즌",
                  "순위결정전": "정규시즌",
                  "순위결정정": "정규시즌",   # 오타까지 함께 처리
                  '조별리그' : "정규시즌",
                  "0": "정규시즌"})
        # ➋ 라운드 표기(1R ~ 33R 등) → 정규시즌
        .str.replace(r"^\d+R$", "정규시즌", regex=True)
)

df["match_type"] = (
    df["match_type"]
        .replace({'와일드카드':"포스트시즌",
                '준플레이오프':"포스트시즌", 
                '플레이오프':"포스트시즌", 
                '한국시리즈':"포스트시즌",
                '파이널 라운드A':"포스트시즌",
                '파이널 라운드B':"포스트시즌",
                '챔피언결정전':"포스트시즌", 
                '준결승':"포스트시즌", 
                '결승':"포스트시즌"})
)
# 결과 확인
print(df["match_type"].unique())




['시범경기' '정규시즌' '올스타전' '포스트시즌']


In [8]:
# 중복성 제거
dup_rows = df.duplicated().sum()
dup_keys = df.duplicated(subset=["game_date","region","game_id"], keep=False).sum()

print(dup_rows)
df.info()


0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3577 entries, 0 to 3576
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   game_id         3577 non-null   int64  
 1   stadium_code    3577 non-null   object 
 2   sports_type     3577 non-null   object 
 3   game_date       3577 non-null   object 
 4   day_of_week     3577 non-null   object 
 5   is_holiday      3577 non-null   int64  
 6   start_time      3577 non-null   object 
 7   home_team_win   3577 non-null   int64  
 8   match_type      3577 non-null   object 
 9   audience        3577 non-null   int64  
 10  region          3577 non-null   object 
 11  accident_count  3577 non-null   float64
 12  injury_count    3577 non-null   float64
 13  death_count     3577 non-null   float64
 14  temperature     3577 non-null   float64
 15  precipitation   3577 non-null   float64
 16  snow_depth      3577 non-null   float64
dtypes: float64(6), int64(4), object

In [11]:
# 범주형 Cardinality & 정제
for col in ["sports_type", "day_of_week", "region", "match_type"]:
    print(df[col].value_counts(dropna=False).head(20), "\n")


sports_type
야구      2026
농구       563
축구       425
배구       282
여자배구     281
Name: count, dtype: int64 

day_of_week
토    744
일    730
금    508
화    505
목    493
수    478
월    119
Name: count, dtype: int64 

region
서울 송파구     511
경기 수원시     403
경남 창원시     279
대전 중구      276
부산 동래구     246
광주 북구      205
인천 미추홀구    201
대구 수성구     185
서울 구로구     178
대구 동구       90
서울 중구       78
경기 화성시      73
경북 김천시      58
울산 중구       58
경기 안양시      58
강원 원주시      55
경기 고양시      54
전북 전주시      51
울산 남구       50
경기 안산시      41
Name: count, dtype: int64 

match_type
페넌트레이스      1851
정규시즌        1025
시범경기         118
플레이오프         60
파이널 라운드B      30
챔피언결정전        26
파이널 라운드A      25
조별리그          24
0             21
29R           12
19R           12
18R           12
16R           12
5R            11
2R            11
한국시리즈         11
33R           11
31R           11
30R           11
1R            11
Name: count, dtype: int64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3577 entries, 0 to 3576
Data

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3577 entries, 0 to 3576
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   game_id         3577 non-null   int64  
 1   stadium_code    3577 non-null   object 
 2   sports_type     3577 non-null   object 
 3   game_date       3577 non-null   object 
 4   day_of_week     3577 non-null   object 
 5   is_holiday      3577 non-null   int64  
 6   start_time      3577 non-null   object 
 7   home_team_win   3577 non-null   int64  
 8   match_type      3577 non-null   object 
 9   audience        3577 non-null   int64  
 10  region          3577 non-null   object 
 11  accident_count  3577 non-null   float64
 12  injury_count    3577 non-null   float64
 13  death_count     3577 non-null   float64
 14  temperature     3577 non-null   float64
 15  precipitation   3577 non-null   float64
 16  snow_depth      3577 non-null   float64
dtypes: float64(6), int64(4), object(7

In [13]:
num_cols = ["accident_count","injury_count","death_count"]
desc = df[num_cols].describe().T   # min-max, IQR
q1,q3 = df["accident_count"].quantile([.25,.75])
iqr   = q3-q1
out   = df[(df["accident_count"] < q1-1.5*iqr) | (df["accident_count"] > q3+1.5*iqr)]

In [16]:
# 상관관계·다중공선성 확인
corr = df.corr(numeric_only=True)
target_corr = corr["accident_count"].sort_values(ascending=False)
# print(corr)
print(target_corr)

accident_count    1.000000
injury_count      0.939950
death_count       0.100951
game_id           0.027119
snow_depth        0.025671
home_team_win    -0.001627
temperature      -0.002274
precipitation    -0.031135
is_holiday             NaN
audience               NaN
Name: accident_count, dtype: float64


In [17]:
from ydata_profiling import ProfileReport
ProfileReport(df, title="1차 데이터셋 품질 리포트").to_file("eda_quality_report.html")

ModuleNotFoundError: No module named 'ydata_profiling'